In [7]:
import keras
import numpy as np
import os
import pickle
import gzip
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [8]:
step_count = 6
base_path = 'C:/Users/abdul/Desktop/forest_python/'
data_path = os.path.join(base_path, 'data')
pickle_folder = os.path.join(data_path, 'pickles')
trips_file_path = os.path.join(data_path, 'map_matched_all/')
mapped_dict_file_name = 'mapped_dict.json'
nodes_file_name = 'chengdu_nodes.json'

In [9]:
def gen_weights(Y_train, sig=False, relu=False):
    weights = {}
    for y in Y_train:
        weights[y] = weights.get(y, 0) + 1
    if sig: return sigmoid_weights(weights)
    if relu: return relu_weights(weights)
    return weights

def relu_weights(weights):
    weights_relu = {}
    low = 50
    for n, w in weights.items():
        if w > low:
            weights_relu[n] = w
        else:
            weights_relu[n] = -999.0

def sigmoid_weights(weights):
    import math
    weights_sigmoid = {}
    for n, w in weights.items():
            weights_sigmoid[n] = 1 / (1 + math.exp(-w))
    return weights_sigmoid

In [10]:
def print_accuracy(dtree, X_test, Y_test, step):
    total_acc = 0
    for i in range(10):
        min_index = round((i * 0.1) * len(Y_test))
        max_index = round((i+1) * 0.1 * len(Y_test))
        x = X_test[min_index:max_index]
        y = Y_test[min_index:max_index]
        pred = dtree.predict(x)
        total_acc += accuracy_score(y, pred)
    total_acc /= 10.0
    print(f'Accuracy for step {step}: {total_acc}')

In [11]:
def train_and_save(step, X, Y):
    X_train = X
    Y_train = Y
    weights = gen_weights(Y_train, sig=True)
    dtree2 = DecisionTreeClassifier(class_weight=weights)
    dtree2.fit(X_train, Y_train)
    fp=gzip.open(os.path.join(pickle_folder, f'dtree_{step}.pkl'),'wb')
    pickle.dump(dtree2,fp)
    fp.close()

In [12]:
def train_save_all():
    for step in range(1, step_count+1):
        pickle_x = os.path.join(pickle_folder, f'X_{step}.pkl')
        pickle_y = os.path.join(pickle_folder, f'Y_{step}.pkl')
        X: np.ndarray = pickle.load(open(pickle_x, "rb")).astype('int16')
        Y: np.ndarray = pickle.load(open(pickle_y, "rb")).astype('int16')
        train_and_save(step, X, Y)

In [16]:
def load_predict(step):
    pickle_x = os.path.join(pickle_folder, f'X_{step}.pkl')
    pickle_y = os.path.join(pickle_folder, f'Y_{step}.pkl')
    X: np.ndarray = pickle.load(open(pickle_x, "rb")).astype('int16')[:2000]
    Y: np.ndarray = pickle.load(open(pickle_y, "rb")).astype('int16')[:2000]
    fp=gzip.open(os.path.join(pickle_folder, f'dtree_{step}.pkl'),'rb')
    dtree=pickle.load(fp)
    fp.close()
    print_accuracy(dtree, X, Y, step)
    

In [84]:
# def load_dtree(step: int) -> DecisionTreeClassifier:
#     fp = gzip.open(os.path.join(pickle_folder, f'dtree_{step}.pkl'), 'rb')
#     dtree = pickle.load(fp)
#     fp.close()
#     return dtree

feature = [np.array([0,0,0,3,4,5,6,9])]
print(dt.predict(feature))
p = dt.predict_proba(feature).tolist()
sp = None
for _p in p:
    if not sp: sp = np.add(_p, 0)
    else: sp = np.add(sp, _p)
dt.classes_[np.argmax(sp)]
# s = dict(zip(dt.classes_, ))

[1502]


1502

In [79]:
np.add(np.array([0,0,0,3,4,5,6,9]), 2)

array([ 2,  2,  2,  5,  6,  7,  8, 11])

In [46]:
'''
15=n, 70=depth 500k in, sigmod
Accuracy for step 1: 0.84487
Accuracy for step 2: 0.7530699999999999
Accuracy for step 3: 0.67774
Accuracy for step 4: 0.6162099999999999
Accuracy for step 5: 0.5616700000000001
Accuracy for step 6: 0.51528

20=n, 80=depth
Accuracy for step 1: 0.8244
Accuracy for step 2: 0.731925
Accuracy for step 3: 0.657875
Accuracy for step 4: 0.5936250000000001
Accuracy for step 5: 0.5365
Accuracy for step 6: 0.49605000000000005'''

'''
n_estimators=30, max_depth=70, sigmoid
Accuracy for step 1: 0.8217000000000001
Accuracy for step 2: 0.734675
Accuracy for step 3: 0.6575
Accuracy for step 4: 0.5940000000000001
Accuracy for step 5: 0.5400499999999999
Accuracy for step 6: 0.490725'''

'\nn_estimators=30, max_depth=70, sigmoid\nAccuracy for step 1: 0.8217000000000001\nAccuracy for step 2: 0.734675\nAccuracy for step 3: 0.6575\nAccuracy for step 4: 0.5940000000000001\nAccuracy for step 5: 0.5400499999999999\nAccuracy for step 6: 0.490725'

In [ ]:
'''
No weights applied
Accuracy for step 1: 0.8217000000000001
Accuracy for step 2: 0.7322750000000001
Accuracy for step 3: 0.6553000000000001
Accuracy for step 4: 0.593425
Accuracy for step 5: 0.5380499999999999
Accuracy for step 6: 0.49262500000000004

SIGMOID:
Accuracy for step 1: 0.8404
Accuracy for step 2: 0.736125
Accuracy for step 3: 0.6582250000000001
Accuracy for step 4: 0.59445
Accuracy for step 5: 0.5404
Accuracy for step 6: 0.497725

RELU:
Accuracy for step 1: 0.828
Accuracy for step 2: 0.7473500000000001
Accuracy for step 3: 0.6609499999999999
Accuracy for step 4: 0.5929000000000001
Accuracy for step 5: 0.54105
Accuracy for step 6: 0.49315
'''